Import's básicos

In [ ]:
import pprint
from json import loads
from IPython.display import display

from pyspark import SparkContext
sc = SparkContext()

criação de um RDD atraves de uma collection

In [ ]:
numbers = sc.parallelize(range(0, 1000000))

In [ ]:
numbers.count()

criação atraves de um arquivo texto:

### estrutura

```javascript
[{  
   "track_disc_number":1,
   "album_id":"7vEJAtP3KgKSpOHVgwm3Eh",
   "track_lyrics":"Love, love me do\nYou know I love you\nI'll always be true\nSo please, love me do\nWhoa, love me do\n\nLove, love me do\nYou know I love you\nI'll always be true\nSo please, love me do\nWhoa, love me do\n\nSomeone to love\nSomebody new\nSomeone to love\nSomeone like you\n\nLove, love me do\nYou know I love you\nI'll always be true\nSo please, love me do\nWhoa, love me do\n\nLove, love me do\nYou know I love you\nI'll always be true\nSo please, love me do\nWhoa, love me do\nYeah, love me do\nWhoa, love me do\nYeah, love me do",
   "track_duration":140173,
   "album_type":"album",
   "track_id":"4aX3n1Pn6jiCbvqBL8eqyP",
   "track_name":"Love Me Do",
   "track_explicit":false,
   "album_image":"https://i.scdn.co/image/9ecfdf528562cae879748b73bd81b64dfa3d5704",
   "track_number":1,
   "album_name":"1",
   "artist_name":"The Beatles"
}]
```

### map
aplica uma **function** a cada elemento do RDD


In [ ]:
lyrics = sc.textFile('artists.json')
rdd = lyrics.map(loads)

Quantas musicas beatles?

In [ ]:
rdd.count()

Vamos olhar como nosso RDD está!

In [ ]:
rdd.collect()

Vamos distinguir todas os nomes de música!

In [ ]:
rdd.map(lambda l: l['track_name']).distinct().collect()

### flatMap

funciona como o **map**, porem ao final da operaçao, os elementos são **flattening** (achatamento)

```javascript
(("Love","Me","Do"),("In", "my", "Life")) = ("Love", "Me", "Do", "In", "my", "Life")
```

In [ ]:
def tokenize(lyric):
    return lyric['track_name'].split()

beatles = rdd.flatMap(tokenize)

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.map(lambda word: (word, 1))

In [ ]:
beatles.collect()

Mergea os valores de cada chave.
```javascript
(("Love",1), ("Girl", 1), ("Girl", 1)) = (("Love",1), ("Girl", 1))
```

In [ ]:
beatles = beatles.reduceByKey(lambda a, b: a + b)

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.map(lambda x:(x[1],x[0]))

In [ ]:
beatles.collect()

In [ ]:
beatles = beatles.sortByKey(ascending=False)

In [ ]:
beatles.collect()

In [ ]:
rdd.filter(lambda lyric: 'love' in lyric['track_name'].lower()).count()

Um **SQLContext** é usado para criação de **DataFrame**, registra dataframes como tabelas, execução de SQL e leitura de aquivos textuais.

In [ ]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)

In [ ]:
df_json = sql.jsonRDD(lyrics)

In [ ]:
df_json.show()

In [ ]:
df_json.printSchema()

In [ ]:
df_beatles = sql.read.json('artists.json')

In [ ]:
df_beatles.printSchema()

In [ ]:
df_beatles.registerTempTable('beatles')

In [ ]:
df_beatles.columns

um dataframe do **spark** é diferente de um dataframe do **pandas**, mesmo que os conceitos sejam muito parecidos.
um dataframe do spark pode ser converdido facilmente em um dataframe pandas.

In [ ]:
df_beatles.toPandas()

In [ ]:
df_beatles.map(lambda l: l.album_name).collect()

In [ ]:
df_beatles.filter("track_duration >  300000").show()

In [ ]:
df_beatles.filter(df_beatles.track_duration >  250000).show()

In [ ]:
sql.sql('select count(*) as count from beatles').show()

In [ ]:
sql.sql('select album_name, count(track_name) as count from beatles group by album_name order by count').show(n=100)

In [ ]:
from pyspark.sql import functions as F
df_beatles \
    .groupBy(df_beatles.album_name)\
    .count()\
    .orderBy("count") \
    .show(n=100)

In [ ]:
df_beatles.toJSON().collect()